# Protein LM Embeddings and Logits as Features

One of the best ways you can leverage a large language model is for feature generation. The internal, numeric representations the neural-net uses to make predictions can be output and used for downstream machine learning tasks. The numeric vectors from NLP models often encode additional, powerful information beyond simple one-hot encodings. Usually feature engineering for biology is heavily task-specific, but in this case the embeddings can be used for a variety of classification, regression, and other tasks.

On the backend, the process involves passing input sequences into the pre-trained model for tokenization and representation of the protein through its neural-net layers. Multiple representations of a protein - numeric vectors and/or matrices, such as attention maps - are created. Here we will quickly demo using ESM2 via GPU-backed REST API in order to quickly transform a sequence into embeddings without installing packages, setting up a GPU, and downloading the model.

In [ ]:
from helpers import api_caller

BIOLMAI_TOKEN = " "  # !!! YOUR API TOKEN HERE !!!

In [2]:
# FOR IN-BROWSER JUPYTER-LITE ONLY #
import micropip  # Install with `pip install pandas` if running notebook elsewhere
await micropip.install('pandas')
await micropip.install('xgboost')
await micropip.install('scikit-learn')
await micropip.install('matplotlib')
await micropip.install('seaborn')

In [3]:
import os

import pandas as pd

from glob import glob

In [4]:
# Load sequences and fluorescence data
df = pd.concat([
    pd.read_csv(f) for f in
    glob(os.path.join('.', 'data', 'protein', 'data', 'fluor*.train.csv'))
])

df.drop_duplicates('seq', inplace=True)

In [5]:
df.shape

(21446, 2)

In [6]:
# SAMPLE some of the data
df_orig = df  # Retain original DF as `df_orig`
df = df_orig.copy().sample(50, random_state=42)

df.head(6)

,seq,label
19035,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.603359
20206,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATCGKLTLKFI...,3.940364
7499,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGVGDATYGKLTLKFI...,3.622829
1077,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,1.594085
18115,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,1.302041
4871,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,1.470378


Here we have a DataFrame containing sequences and their measured fluorescence. We can use the ESM2 embeddings as features to perform a quick regression to predict fluorescence values; but first we need to actually generate the embeddings.

Let's write a function that takes a sequence and requests its embeddings via REST API. The [ESM2 encode endpoint documentation](https://api.biolm.ai/#daa50ec7-0da2-4bff-ab4c-3ead7f377154) provides examples of the structure of a response.

In [7]:
SLUG = 'esm2-650m'
ACTION = 'encode'

## Single Sequence Embeddings

In [8]:
test_protein = df.sample(1).seq.iloc[0]

print("Sequence length: {}\n{}".format(len(test_protein), test_protein))

Sequence length: 237
SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIGDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLVFVTTAGITHGMDELYK


We can POST that sequence:

In [9]:
# Definition comes from API reference docs linked just above
data = {
        "params": {
            "include": [
                "mean",
                "contacts",
                "logits",
                "attentions"
            ]
    },
  "items": [
    {
      "sequence": test_protein
    }
  ]
}

# Async function to get embeddings via REST API
r = await api_caller(
    SLUG,
    ACTION,
    data,
    TOKEN
)

In [10]:
from IPython.display import JSON

JSON(r)

<IPython.core.display.JSON object>

We get back a list of dicts. Each dictionary item `mean_representations` contains the mean representations of a layer(s) from ESM2. In this case, we return the embeddings from the the final hidden layer, `33`.

Let's load this representation and look at its shape.

In [11]:
embed_single = pd.DataFrame(r['results'][0]['mean_representations']['33']).T

embed_single

<class 'KeyError'>: 'predictions'

We can see that while the original sequence is `237` residues, the LLM uses a vector of `1280` to represent sequences. So, anytime we request an embedding for a sequence, we'll get back a representation that is the same size as another sequence. This makes downstream ML, especially with other NNs, nice and easy since we don't have to worry about padding.

Let's get the embeddings for all sequences in the DF, asynchronously, via REST API.

## Embeddings for All Seqs in DF

In [12]:
import asyncio
import random

# Limit concurrency to 3 requests at a time (*not* RPS)
sem = asyncio.Semaphore(5)


results = []

async def f(seq):
    # Add small jitter
    jitter = random.randint(1, 100) / 100.0  
    await asyncio.sleep(jitter * 4)
    # Definition comes from API reference docs linked just above
    data = {
        "params": {
            "include": [
                "mean",
                "contacts",
                "logits",
                "attentions"
            ]
        },
        "items": [
        {
          "sequence": seq

        }
      ]
    }
    # Request the embeddings
    async with sem:
        r = await api_caller(
            SLUG,
            ACTION,
            data,
            TOKEN
        )
        res = r['results'][0]['mean_representations']['33']
        
    return res


ops = [f(seq) for seq in df['seq'].to_list()]
rets = await asyncio.gather(*ops)

<class 'KeyError'>: 'predictions'

In [13]:
len(rets)

<class 'NameError'>: name 'rets' is not defined

Now, instead of getting the embeddings, we could have written a function to retrieve the logits. In fact, they were returned by the same API endpoint - we simply need to use a different key. We can look at the single-sequence API request we made earlier to find them:

In [14]:
logits_single = pd.DataFrame(r['results'][0]['logits']).T

logits_single

<class 'KeyError'>: 'predictions'

One could also use the sum or mean as a representation of sequence:

In [15]:
logits_single.iloc[0].sum()

<class 'NameError'>: name 'logits_single' is not defined

In [16]:
logits_single.iloc[0].mean()

<class 'NameError'>: name 'logits_single' is not defined

## Modeling with XGBoost

In order to not require any transformations of the data, let's use a tree-based method to quickly create a regression model using these ESM2 embeddings.

We'll see how well we can model fluorescence with about 1,000 samples. With so few, we should make sure to uniformly sample our labels as much as possible.

This will take several minutes, depending on your download speed and connection bandwidth. We've added a slight jitter and limit of 5 concurrent connections above, to prevent thundering herds and overwhelming your connection. You can open your browser's Developer Tools (aka Inspector) and watch the traffic under the Network tab.

In [17]:
df_orig['bins'] = pd.cut(df_orig.label, bins=100)
df_orig['bins'] = df_orig['bins'].astype('str')

df_orig.bins.value_counts()

(1.281, 1.312]    1861
(3.669, 3.697]    1494
(3.612, 3.64]     1330
(3.697, 3.726]    1255
(3.64, 3.669]     1248
                  ... 
(2.448, 2.476]       2
(2.646, 2.675]       2
(2.05, 2.079]        2
(2.419, 2.448]       1
(4.066, 4.095]       1
Name: bins, Length: 97, dtype: int64

In [18]:
sampled = []

df_orig = df_orig.sample(df_orig.shape[0], random_state=42)

for n, grp in df_orig.groupby('bins'):
    # At most 15 rows from each bin
    samp = grp.head(10).reset_index(drop=True)
    sampled.append(samp)
    
sampled_rows = pd.concat(sampled, axis=0).reset_index(drop=True)

df_xgboost = sampled_rows

df_xgboost.shape

(810, 3)

In [19]:
ops = [f(seq) for seq in df_xgboost['seq'].to_list()]
xgboost_embeddings = await asyncio.gather(*ops)

<class 'KeyError'>: 'predictions'

In [20]:
embeddings = pd.DataFrame(xgboost_embeddings)

embeddings.shape

<class 'NameError'>: name 'xgboost_embeddings' is not defined

In [21]:
await micropip.install('seaborn')

In [22]:
from sklearn import model_selection

# Create 80:20 train:test split
train_x, test_x, train_y, test_y = model_selection.train_test_split(
    embeddings,
    df_xgboost.label,
    test_size=0.2,
    random_state=54
)

print("X Train size: {}\nX Test size: {}".format(train_x.shape, test_x.shape))
print("Y Train size: {}\nY Test size: {}".format(train_y.shape, test_y.shape))

<class 'NameError'>: name 'embeddings' is not defined

In [23]:
embeddings.head(3)

<class 'NameError'>: name 'embeddings' is not defined

In [24]:
import xgboost as xgb

#Set up cross-validation modeling objective
data_dmatrix = xgb.DMatrix(data=train_x, label=train_y)

params = {
    'booster': 'gbtree',
    "objective": "reg:squarederror",
    'colsample_bytree': 0.40,
    'learning_rate': 0.2,
    'max_depth': 40,
    'eval_metric': 'rmse',
    'alpha': 0.8,
}

# Run CV
cv_results = xgb.cv(
    dtrain=data_dmatrix,
    params=params,
    nfold=5,
    num_boost_round=100,
    early_stopping_rounds=6,
    metrics="rmse",
    as_pandas=True,
    seed=42
)

<class 'NameError'>: name 'train_x' is not defined

We can see how the performance of the model training started...

In [25]:
cv_results.head()

<class 'NameError'>: name 'cv_results' is not defined

In [26]:
cv_results.tail(10)

<class 'NameError'>: name 'cv_results' is not defined

In [27]:
# Final RMSE on test set
print((cv_results["test-rmse-mean"]).tail(1))

<class 'NameError'>: name 'cv_results' is not defined

In [28]:
# Final SD on test set
print((cv_results["test-rmse-std"]).tail(1))

<class 'NameError'>: name 'cv_results' is not defined

We can get context for these values by looking at the `Y` values that were used in this cross-validation experiment.

In [29]:
from matplotlib import pyplot as plt

plt.figure(figsize=(6, 5))

train_y.hist()

<Figure size 600x500 with 0 Axes>

<class 'NameError'>: name 'train_y' is not defined

Now we can train a model with the cross-validated parameters, using our full training dataset.

In [30]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=80)

<class 'NameError'>: name 'params' is not defined

Measuring the predicted values against our test set, let's see how well the model did using just the sequence embeddings features.

In [31]:
y_pred = xg_reg.predict(xgb.DMatrix(data=test_x, label=test_y))

<class 'NameError'>: name 'xg_reg' is not defined

In [32]:
import seaborn as sns

sns.regplot(x=test_y, y=y_pred)

<class 'NameError'>: name 'test_y' is not defined

Lastly, we can attempt to inspect the model and learn a bit more about it, its fit, and our data.

In [33]:
xgb.plot_importance(xg_reg, max_num_features=20, grid=False)
plt.rcParams['figure.figsize'] = [10, 8]
plt.show()

<class 'NameError'>: name 'xg_reg' is not defined